# Building a book of the Julich-Brain probabilistic cytoarchitectonic atlas

In [ ]:
import siibra
from nilearn import plotting
from anytree import Node, RenderTree, ContRoundStyle

In [ ]:
atlas = siibra.atlases['human']

We start the book with a print of a tree of mapped regions

In [ ]:
jubrain = atlas.get_parcellation('julich 2.9')
r = atlas.get_region('123 left')

In [ ]:
def build_path(regions,parent=None):
    if len(regions)==0:
        return
    if len(regions)>0:
        result = Node(regions[0].name, parent=parent)
    if len(regions)>1:
        build_path(regions[1:], parent=result)
    return result

In [ ]:
siibra.set_ebrains_token("eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJfNkZVSHFaSDNIRmVhS0pEZDhXcUx6LWFlZ3kzYXFodVNJZ1RXaTA1U2k0In0.eyJleHAiOjE2MzA5MTI5NTksImlhdCI6MTYzMDkxMTE1OSwiYXV0aF90aW1lIjoxNjMwOTExMTU4LCJqdGkiOiJlNjI3NzViNS03YTBmLTQ1MTgtYmU0MS1mMDcyYjg3ZWQ3NzUiLCJpc3MiOiJodHRwczovL2lhbS5lYnJhaW5zLmV1L2F1dGgvcmVhbG1zL2hicCIsImF1ZCI6WyJyZWFsbS1tYW5hZ2VtZW50IiwianVweXRlcmh1YiIsInh3aWtpIiwidGVhbSIsImdyb3VwIl0sInN1YiI6IjY1MDM1MWJmLTI0OGUtNGUzYy04MWNjLTQ0MGZmNmQ5ZjQxNCIsInR5cCI6IkJlYXJlciIsImF6cCI6ImtnIiwibm9uY2UiOiJmZTRiNjJkMi1iNGRlLTQ4ZGQtODE4Ny0wYzIxZTY2Mjc4Y2EiLCJzZXNzaW9uX3N0YXRlIjoiY2YzZWM2YzMtNjY5Yy00YmIxLWFkMjgtMDg4ODEwYmE1ZjQ1IiwiYWNyIjoiMSIsInNjb3BlIjoib3BlbmlkIHByb2ZpbGUgcm9sZXMgZW1haWwgdGVhbSIsImVtYWlsX3ZlcmlmaWVkIjp0cnVlLCJnZW5kZXIiOiIwIiwibmFtZSI6IlRpbW8gRGlja3NjaGVpZCIsInByZWZlcnJlZF91c2VybmFtZSI6ImRpY2tzY2hlIiwiZ2l2ZW5fbmFtZSI6IlRpbW8iLCJmYW1pbHlfbmFtZSI6IkRpY2tzY2hlaWQiLCJlbWFpbCI6InQuZGlja3NjaGVpZEBmei1qdWVsaWNoLmRlIn0.wJpLNVsn7DAfimAzCx9NB31P2ArLxa-EM0jBIH5KzognipRvI05nebBBAbNHmnUPNySvYtXsdiKuN_m5sGMH2OO_ZBMFOXRQby_pzborl-9QLLXORVDv_I0ji7KjWHBa7h-8PPpPqO4MSXQB0FZ9wyOaOFnXFpLIXNbFGwgukt7eSv_SwxHIVnONXH_MQ9Ggk0E5PpfBItLI4XCkRnGC_IXxPHBWi12qx8vm98Cij-DKkrAJ8KxTDv243_VBE836s-NasyaQMXFQ-51t_IGnOPekNkz5b2PRBFAZX6d_v8cx_tXEmdxTbJyMIk_i9ofGMDoDUFMfdHaiMCwXSC6aGw")

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use('Agg')

for region in {r.parent for r in jubrain if r.is_leaf}:

    descriptions = region.descriptions
    citations = [p['citation'] for p in region.publications]
    pmaps = {}
    mpms = {}

    for i,child in enumerate(region.children):

        # print the probabilistic map
        plt.figure()
        m = child.get_regional_map("mni152","continuous")
        plotting.plot_stat_map(m.fetch(), axes=plt.gca(), display_mode="ortho", colorbar=False)
        plt.gca().axis(False)
        pmaps[child] = {'file':f"PMAP_{child.key}.png", 'url':m.url}
        plt.tight_layout()
        plt.savefig(pmaps[child]['file'], dpi=150)
        
        # print the maximum probability map
        plt.figure()
        m = child.build_mask("mni152",maptype="labelled")
        plotting.plot_roi(m, axes=plt.gca(), display_mode="ortho")
        plt.gca().axis(False)
        mpms[child] = {'file':f"MPM_{child.key}.png"}
        plt.tight_layout()
        plt.savefig(mpms[child]['file'], dpi=150)
        
        descriptions.extend(child.descriptions)
        for key in ['cite','citation']:
            citations.extend(
                p[key] for p in child.publications 
                if key in p)    

    with open(f"{region.key}.md",'w') as f:
        
        f.write("---\ngeometry: margin=3cm\n---\n\n")
        
        f.write(f"# {region.name}\n\n")
        tree = "\n".join(
            "%s%s" % (pre, node.name)
            for pre, _, node in RenderTree(build_path(region.ancestors))
        )
        f.write(f"```\n{tree}\n```")
        f.write("\n\n")

        f.write("### Probability maps\n\n")
        f.write(" ".join(f'![]({i["file"]}){{ width="50%" }}' 
                         for i in pmaps.values()))
        f.write("\n")
        f.write("Download:\n\n")
        for r,i in pmaps.items():
            f.write(f'- [{r.name}]({i["url"]})\n')   
        f.write("\n\n")
        
        f.write("### Maximum probability maps\n\n")
        f.write(" ".join(f'![]({i["file"]}){{ width="50%" }}' 
                         for i in mpms.values()))
        f.write("\n\n")
        
        
        f.write("### Description\n\n")
        descriptions = list(set(descriptions))
        f.write("\n".join(descriptions))
        f.write("\n\n")
        
        f.write("### Publications\n\n")
        citations = list(set(citations))
        f.write("\n".join(citations))
        f.write("\n\n")
        
        if len(descriptions)>0:
            break
            